Компания "Цифры" разрабатывает решения для эффективной работы промышленных предприятий.

Задача: Подготовить прототип модели машинного обучения. Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. 

Этапы:
1. Подготовка данных;
2. Исследовательский анализ данных;
3. Построение и обучение модели.

Описание данных

Технологический процесс:
- Rougher feed — исходное сырье
- Rougher additions (или reagent additions) — флотационные реагенты: Xanthate, Sulphate, Depressant
  - Xanthate **— ксантогенат (промотер, или активатор флотации);
  - Sulphate — сульфат (на данном производстве сульфид натрия);
  - Depressant — депрессант (силикат натрия).
- Rougher process (англ. «грубый процесс») — флотация
- Rougher tails — отвальные хвосты
- Float banks — флотационная установка
- Cleaner process — очистка
- Rougher Au — черновой концентрат золота
- Final Au — финальный концентрат золота

Параметры этапов:
- air amount — объём воздуха
- fluid levels — уровень жидкости
- feed size — размер гранул сырья
- feed rate — скорость подачи

# Подготовка данных

Загружаю файлы


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [ ]:
try:
    df_full = pd.read_csv('/datasets/gold_recovery_full_new.csv')
    df_train = pd.read_csv('/datasets/gold_recovery_train_new.csv')
    df_test = pd.read_csv('/datasets/gold_recovery_test_new.csv')
except:
    from google.colab import drive
    drive.mount('/content/drive')
    df_full = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/12_sprint/gold_recovery_full_new.csv')
    df_train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/12_sprint/gold_recovery_train_new.csv')
    df_test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/12_sprint/gold_recovery_test_new.csv')

Задаю гиперпараметры

In [ ]:
RANDOM_STATE = 12345

Анализ загруженных данных.

In [ ]:
df_full.head(5)

In [ ]:
df_full.info()

In [ ]:
df_train

In [ ]:
df_train.info()

In [ ]:
df_test

In [ ]:
df_test.info()

### Проверка правильности расчёта эффективности обогащения

Проверка правильности расчёта обогащения сырья.

In [ ]:
C = df_train['rougher.output.concentrate_au'] #доля золота в концентрате после флотации/очистки
F = df_train['rougher.input.feed_au'] #доля золота в сырье/концентрате до флотации/очистки
T = df_train['rougher.output.tail_au'] #доля золота в отвальных хвостах после флотации/очистки

Расчёт эффективности обогащения

In [ ]:
recovery = ( (C * (F-T))/ (F * (C - T)) ) *100

Сравнение расчитанной эффективности обучения с предоставленными в данных.

In [ ]:
mae_recovery = mean_absolute_error(df_train['rougher.output.recovery'], recovery)
mae_recovery

Разница между полученным результатом и предоставленным в исходных данных минимальна. Можно считать что эффективность изначально была расчитана корректно.

### Сравнение данных в тестовом и полном наборе данных.

Сравниваю данные в файле с полным датасетом и тестовым

In [ ]:
test_columns = list(df_test.columns.values)
full_columns = list(df_full.columns.values)
not_in_full =[]
for i in full_columns:
    if i not in test_columns:
        not_in_full.append(i)

print(pd.Series(not_in_full))


В тестовом файле отсутсвуют колонки содержащие значения количества веществ содержащихся на выходе после каждого этапа. 

### Предобработка данных

Обработаю данные для дальнейшей работы. Ранее в данных были обнаружены пропуски, заменим их близкими по значению. В ТЗ сказано, что соседние по времени значения схожи.

In [ ]:
df_full[['rougher.output.concentrate_au','final.output.concentrate_au']].isna().sum()

In [ ]:
df_full = df_full.fillna(method='ffill') 
df_train = df_train.fillna(method='ffill')
df_test = df_test.fillna(method='ffill')

Проверяю

In [ ]:
df_full.info()

In [ ]:
df_train.info()

Изменяю тип данных в столбцах с датой

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_full['date'] = pd.to_datetime(df_full['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

Проверка

In [ ]:
print(type(df_test['date'][0]))
print(type(df_full['date'][0]))
print(type(df_train['date'][0]))

Проверяю дубликаты

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_test.duplicated().sum()

In [ ]:
df_full.duplicated().sum()

Промежуточные выводы:
1. Загруженны данные.
2. В данных обнаружены пропуски. Они были заменены схожими на основе данных близких по времени.
3. Вывлено различие в количесвте признаков между тестовой и общей выборкой.

# Подготовка прототипа модели МО

### Анализ изменения концентрации AU, AG, PB на разных этапах отчистки.

Исследуем измененя уровня содержания золота на всех этапах отчистки отдельно и общую динамику.

In [ ]:
df_full['rougher.output.concentrate_au'].hist(figsize=(15,7), bins = 30,alpha=0.5, legend=True)
plt.title('Концентрация золота после флотации'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


df_full['primary_cleaner.output.concentrate_au'].hist(figsize=(15,7), bins = 30,alpha=0.5, legend=True)
plt.title('Концентрация золота после первичной отчистки'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


df_full['final.output.concentrate_au'].hist(figsize=(15,7), bins = 30,alpha=0.5, legend=True)
plt.title('Концентрация золота в финальном продукте'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


Рассмотрим содержание серебра на всех этапах отчистки сырья.

In [ ]:
df_full['rougher.output.concentrate_ag'].hist(figsize=(15,7), bins = 30,alpha=0.5, legend=True)
plt.title('Концентрация серебра после флотации'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


df_full['primary_cleaner.output.concentrate_ag'].hist(figsize=(15,7), bins = 30, alpha=0.5, legend=True)
plt.title('Концентрация серебра после первичной отчистки'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


df_full['final.output.concentrate_ag'].hist(figsize=(15,7), bins = 30, alpha=0.5, legend=True)
plt.title('Концентрация серебра в финальном продукте'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


Исследуем содержание свинца на всех этапах отчистки

In [ ]:
df_full['rougher.output.concentrate_pb'].hist(figsize=(15,7), bins = 30,alpha=0.5, legend=True)
plt.title('Концентрация свинца после флотации'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');

df_full['primary_cleaner.output.concentrate_pb'].hist(figsize=(15,7), bins = 30,alpha=0.5, legend=True)
plt.title('Концентрация свинца после первичной отчистки'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


df_full['final.output.concentrate_pb'].hist(figsize=(15,7), bins = 30,alpha=0.5, legend=True)
plt.title('Концентрация свинца в финальном продукте'+ "\n")
plt.xlabel('Доля, %')
plt.ylabel('Кол-во наблюдений');


Рассмотрю в динамике изменения концентрации веществ в сырье. Дополнительно рассмотрим содержание в "хвостах".

In [ ]:
au_on_stages = df_full[['rougher.input.feed_au', 'rougher.output.concentrate_au','primary_cleaner.output.concentrate_au','final.output.concentrate_au']].median()
ag_on_stages = df_full[['rougher.input.feed_ag','rougher.output.concentrate_ag','primary_cleaner.output.concentrate_ag','final.output.concentrate_ag']].median()
pb_on_stages = df_full[['rougher.input.feed_pb','rougher.output.concentrate_pb','primary_cleaner.output.concentrate_pb','final.output.concentrate_pb']].median()

In [ ]:
plt.figure(figsize=(10,5), dpi= 80)
pb_on_stages.plot(kind='line', color="g", label = 'Pb',legend='pb')
au_on_stages.plot(kind='line', color="black", legend="Au", label = 'Au')
ag_on_stages.plot(kind='line', color="dodgerblue", legend="Pg", label = 'Ag')
plt.title('Изменение концентрациии веществ в сырье на этапах отчистки')
plt.show()

In [ ]:
au_tails_on_stages = df_full[['rougher.output.tail_au','primary_cleaner.output.tail_au','secondary_cleaner.output.tail_au', 'final.output.tail_au']].median()
ag_tails_on_stages = df_full[['rougher.output.tail_ag','primary_cleaner.output.tail_ag','secondary_cleaner.output.tail_ag', 'final.output.tail_ag']].median()
pb_tails_on_stages = df_full[['rougher.output.tail_pb','primary_cleaner.output.tail_pb','secondary_cleaner.output.tail_pb', 'final.output.tail_pb']].median()

In [ ]:
plt.figure(figsize=(10,5), dpi= 80)
pb_tails_on_stages.plot(kind='line',color="g", label = 'Pb',legend='pb')
au_tails_on_stages.plot(kind='line',color="black", legend="Au", label = 'Au')
ag_tails_on_stages.plot(kind='line',color="dodgerblue", legend="Pg", label = 'Ag')
plt.title('Изменение концентрациии веществ в "хвостах" на этапах отчистки')
plt.show()

### Анализ размера гранул сырья в тестовой и обучающей выборке.

In [ ]:
test_fraction = df_test[['rougher.input.feed_size','primary_cleaner.input.feed_size']]
train_fraction = df_train[['rougher.input.feed_size','primary_cleaner.input.feed_size']]

In [ ]:
df_test['rougher.input.feed_size'].hist(figsize=(10,5), bins = 30,alpha=0.5,label='test')
plt.title('Размеры гранул в тестовой выборке после флотации'+ "\n")
plt.xlabel('Мкм')
plt.ylabel('Кол-во наблюдений')
plt.xlim(0,150)
plt.legend()

df_train['rougher.input.feed_size'].hist(figsize=(15,7), bins = 30,alpha=0.5, label='train')
plt.title('Размеры гранул в тренировочной выборке после флотации'+ "\n")
plt.xlabel('Мкм')
plt.ylabel('Кол-во наблюдений')
plt.xlim(0,150)
plt.legend();

In [ ]:
df_test['primary_cleaner.input.feed_size'].hist(figsize=(10,5), bins = 30,alpha=0.5, label='test')
plt.title('Размеры гранул в тестовой выборке после первичной отчистки'+ "\n")
plt.xlabel('Мкм')
plt.ylabel('Кол-во наблюдений')
plt.xlim(2,15)
plt.legend()

df_train['primary_cleaner.input.feed_size'].hist(figsize=(15,7), bins = 30,alpha=0.5, label='train')
plt.title('Размеры гранул в тренировочной выборке после первичной отчистки'+ "\n")
plt.xlabel('Мкм')
plt.ylabel('Кол-во наблюдений')
plt.xlim(2,15)
plt.legend();

In [ ]:
print(f'Средний размер гранул в тестовой выборке после флотации {df_test["rougher.input.feed_size"].mean():.2f} мкм.')
print(f'Средний размер гранул в тренировочной выборке после флотации {df_train["rougher.input.feed_size"].mean():.2f} мкм.')
print(f'Средний размер гранул в тестовой выборке после первичной отчистки {df_test["primary_cleaner.input.feed_size"].mean():.2f} мкм.')
print(f'Средний размер гранул в тренировочной выборке после первичной отчистки {df_train["primary_cleaner.input.feed_size"].mean():.2f} мкм.')

Промежуточный вывод:
- Размер гранул после флотации имеют небольшие различия в 4,3 мкм. 
- После первичной отчистки размеры почти идентичны. 
- Значения распределены одинаково. 
- Оценка модели будет правильной.

## Анализ суммарной концентрации всех веществ на разных стадиях.

Выделяю данные содержащие информацию о веществах.

In [ ]:
df_full_sum = df_full

In [ ]:
df_full_sum['rougher_input_sum'] = df_full['rougher.input.feed_ag'] + df_full['rougher.input.feed_au'] + df_full['rougher.input.feed_pb'] + df_full['rougher.input.feed_sol']
df_full_sum['roughter_output_sum'] = df_full['rougher.output.concentrate_ag'] + df_full['rougher.output.concentrate_au'] + df_full['rougher.output.concentrate_pb'] + df_full['rougher.output.concentrate_sol']
df_full_sum['primary_cleaner_output_sum'] = df_full['primary_cleaner.output.concentrate_ag'] + df_full['primary_cleaner.output.tail_au'] + df_full['primary_cleaner.output.tail_pb'] + df_full['primary_cleaner.output.tail_sol']
df_full_sum['final_output_sum'] = df_full['final.output.concentrate_ag'] + df_full['final.output.tail_au'] + df_full['final.output.tail_pb'] + df_full['final.output.tail_sol']



In [ ]:
stages = pd.DataFrame()
stages = df_full[['rougher_input_sum', 'roughter_output_sum', 'primary_cleaner_output_sum', 'final_output_sum']]


In [ ]:
def histbox(df:pd.DataFrame):
    df_hist = df.melt()
    hue = 'variable'
    f, (ax_hist, ax_box) = plt.subplots(2, sharex=True, figsize=(10,10),
                                        gridspec_kw={"height_ratios": (.9,.1)})

    sns.boxplot(data = df, orient="h", ax=ax_box, fliersize=.5)

    sns.histplot(data = df_hist, x="value", ax=ax_hist, hue=hue)
    

    sns.despine(ax=ax_hist)
    sns.despine(ax=ax_box, left=True)
    ax_box.tick_params(axis="y",which="both",labelleft=False)

template = ['rougher_input_{}','roughter_output_{}',
            'primary_cleaner_output_{}','final_output_{}']
func = 'sum'
columns = [txt.format(func) for txt in template]

histbox(df_full[columns])

На гистограме видно нормальное распределение значений. На боксплоте видны аномальные значения. Оценим их объем.

In [ ]:
print('Соотношение аномальных значений в параметрах сырья к общему количеству: ',(df_full_sum['rougher_input_sum']<10).mean())
print('Соотношение аномальных значений сырья к общему количеству после флотации: ',(df_full_sum['roughter_output_sum']<20).mean())
print('Соотношение аномальных значений сырья к общему количеству после первичной обработки: ',(df_full_sum['primary_cleaner_output_sum']<10).mean())
print('Соотношение аномальных значений сырья к общему количеству, финальные харрактеристики: ',(df_full_sum['final_output_sum']<10).mean())

Выбросов мало. Рассмотрим их поподробнее.

In [ ]:
print(df_full_sum[df_full_sum['rougher_input_sum']<5]['rougher_input_sum'])
print()
print(df_full_sum[df_full_sum['roughter_output_sum']<5]['roughter_output_sum'])
print()
print(df_full_sum[df_full_sum['primary_cleaner_output_sum']<3]['primary_cleaner_output_sum'])
print()
print(df_full_sum[df_full_sum['final_output_sum']<5]['final_output_sum'])

Выявлены аномальные значения близкие к нулю или равные ему на разных этапах отчистки. Объем данных небольшой. Можно удалить в выборках для улучшения качества данных и модели.

In [ ]:
df_train_sum = df_train
df_train_sum['rougher_input_sum'] = df_train['rougher.input.feed_ag'] + df_train['rougher.input.feed_au'] + df_train['rougher.input.feed_pb'] + df_train['rougher.input.feed_sol']
df_train_sum['roughter_output_sum'] = df_train['rougher.output.concentrate_ag'] + df_train['rougher.output.concentrate_au'] + df_train['rougher.output.concentrate_pb'] + df_train['rougher.output.concentrate_sol']
df_train_sum['final_output_sum'] = df_train['final.output.concentrate_ag'] + df_train['final.output.tail_au'] + df_train['final.output.tail_pb'] + df_train['final.output.tail_sol']
df_train_sum['primary_cleaner_output_sum'] = df_train['primary_cleaner.output.concentrate_ag'] + df_train['primary_cleaner.output.tail_au'] + df_train['primary_cleaner.output.tail_pb'] + df_train['primary_cleaner.output.tail_sol']


In [ ]:
full_anomaly = df_full_sum.query(" rougher_input_sum <5 |  roughter_output_sum  < 5 | primary_cleaner_output_sum < 3 ")
df_full_clear = df_full.drop(index = full_anomaly.index)
train_anomaly = df_train_sum.query(" rougher_input_sum <5 |  roughter_output_sum  < 5 | primary_cleaner_output_sum < 3 ")
df_train_clear = df_train.drop(index = train_anomaly.index)



Промежуточные выводы:
1. Проведен анализ изменения концентрации золота, серебра и свинца на разных этапах в основном концентрате и в "хвостах":
  - золото - постепенное увеличение; 
  - серебро - постепенное уменьшение;
  - свинец - небольшое увеличение и равенство на заключительных этапах;
  - в "хвостах" содержание данных элементов увеличивается с последующим снижением к финальной обработке.
2. Проанализирован размер гранул в тестовой и обучающей выборке - распределение нормальное, параметры схожи.
3. Проанализирован тренд изменения суммарной концентрации всех веществк на всех этапах. Наблюдается рост после флотации с последующим уменьшением.
4. Рассмотренно распределение суммарной концентрации веществ на каждом этапе - распределение нормальное.
5. При анализе распределения выявлены аномалии. Объём аномалий незначительный, принято решение убрать их из данных.

# Построение модели

## Вычисление итоговой sMAPE

Создам функцию для расчёта итогового smape.

$ \text{sMAPE} = \frac{1}{N}\sum_{i=1}^{N}{\frac{|y_i - \hat{y}_i|}{(|y_i| + |\hat{y}_i|) \div 2}}$
   

In [ ]:
def smape (target, predict):
    smape = 1/len(target) * np.sum(2 * np.abs(predict-target) / (np.abs(target) + np.abs(predict))*100)
    return smape


def smape_total (roughter, final):
    total_smape = 0.25*roughter + 0.75*final
    return total_smape

## Обучение моделей

Подготовим данные и обучим модель для чернового концентрата.
Выбираю столбцы с признаками из тестового датасета.

In [ ]:
columns_test = list(df_test.drop('date',axis=1).columns.values)

Оставляю в обучающей выборке только нужные колонки.

In [ ]:
df_train_model = df_train_clear[columns_test]

Целевой признак возьму из чистого тренировочного датасета. Для предсказания возьмём признкаи из данных по флотации.

In [ ]:
df_train_model.shape

In [ ]:
df_train_scaled = df_train_model.copy()

features = df_train_scaled[columns_test]
scaler = StandardScaler().fit(df_train_scaled.values)
features = scaler.transform(features.values)
df_train_scaled[columns_test] = features

Провожу кросс валидацию посредством kFold.

In [ ]:
kf = KFold(n_splits=5)

Определяю лучший smape на модели Линейной регрессии

In [ ]:
best_smape_total = 100

for train_index, valid_index in kf.split(df_train_model):
    features_train = df_train_scaled.iloc[train_index]
    features_valid = df_train_scaled.iloc[valid_index]

    target_train = df_train.iloc[train_index][['rougher.output.recovery', 'final.output.recovery']]
    target_valid = df_train.iloc[valid_index][['rougher.output.recovery', 'final.output.recovery']]

    model = LinearRegression()      
    model.fit(features_train, target_train) 
    predicts = model.predict(features_valid)
    predicts_rought = predicts[:,0]
    predicts_final = predicts[:,1]

    smape_rought = smape(target_valid['rougher.output.recovery'], predicts_rought)

    smape_final = smape(target_valid['final.output.recovery'], predicts_final)

    total_smape = smape_total (smape_rought, smape_final)

    if total_smape > best_smape_total:
        best_smape_total = total_smape 
        

print( 'Лучший smape на модели линейной регрессии:', best_smape_total)

Определяю лучший smape на модели случайного леса.

In [ ]:
best_smape_total = 100
best_est = 0
best_depth = 0
for train_index, valid_index in kf.split(df_train_model):
    for est in range(1,15,1):
        for depth in range(10,50,1):
            features_train = df_train_scaled.iloc[train_index]
            features_valid = df_train_scaled.iloc[valid_index]

            target_train = df_train.iloc[train_index][['rougher.output.recovery', 'final.output.recovery']]
            target_valid = df_train.iloc[valid_index][['rougher.output.recovery', 'final.output.recovery']]

            model = RandomForestRegressor(n_estimators = est, max_depth = depth, random_state = RANDOM_STATE)   
            model.fit(features_train, target_train) 
            predicts = model.predict(features_valid)
            predicts_rought = predicts[:,0]
            predicts_final = predicts[:,1]

            smape_rought = smape(target_valid['rougher.output.recovery'], predicts_rought)

            smape_final = smape(target_valid['final.output.recovery'], predicts_final)

            total_smape = smape_total (smape_rought, smape_final)

            if total_smape > best_smape_total:
                best_smape_total = total_smape
                best_est = est
                best_depth = depth  
        

print( 'Лучший smape на модели случайного леса:', best_smape_total, ', needed depth:', best_depth, ', needed est:', best_est)

Проверяю модель на тестовой выборке.

Для удобства добавлю столбцы с целевыми праметрами.

In [ ]:
df_test = df_test.dropna()
df_test = df_test.merge(df_full_clear[['rougher.output.recovery', 'final.output.recovery', 'date' ]], on = 'date')
df_test = df_test.drop('date',axis=1)


Подготовка двнных для проверки на тестовых данных

In [ ]:
features_test = df_test.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1)
target_test = df_test[['rougher.output.recovery','final.output.recovery']]

In [ ]:
features_train = df_train[columns_test]
target_train = df_train[['rougher.output.recovery','final.output.recovery']]

In [ ]:
model = RandomForestRegressor(random_state=RANDOM_STATE, max_depth=best_depth, n_estimators=best_est)

model.fit(features_train, target_train) 
predicts = model.predict(features_test)
predicts_rought = predicts[:,0]
predicts_final = predicts[:,1]

smape_rought = smape(df_test['rougher.output.recovery'], predicts_rought)

smape_final = smape(df_test['final.output.recovery'], predicts_final)

total_smape = smape_total (smape_rought, smape_final)

print('sMAPE =', total_smape)

Промежуточные выводы:
1. Создана функция для подсчёта sMAPE.
2. Обучена модель на тренировчных данных.
3. Определена лучшая модель и гиперпараметры.
4. Модель проверена на тренировчных данных.

# Выводы

1. Загруженны данные. В данных обработаны пропуски. 
2. Вывлено различие в количесвте признаков между тестовой и общей выборкой. Это было учтено при дальнейшей работе и обучении моделей.
3. Проверена правильность расчёта эффективности обогащения - расчёты верны.
4. Проведен анализ изменения концентрации золота, серебра и свинца на разных этапах в основном концентрате и в "хвостах":
  - золото - постепенное увеличение; 
  - серебро - постепенное уменьшение;
  - свинец - небольшое увеличение и равенство на заключительных этапах;
  - в "хвостах" содержание данных элементов увеличивается с последующим снижением к финальной обработке.
5. Проанализирован размер гранул в тестовой и обучающей выборке - распределение нормальное, параметры схожи.
6. Проанализирован тренд изменения суммарной концентрации всех веществ на всех этапах. Наблюдается после рост флотации с последующим уменьшением.
7. Рассмотренно распределение суммарной концентрации веществ на каждом этапе - распределение нормальное. При анализе распределения выявлены и обработаны аномалии.
8. Создана функция для подсчета sMAPE необходимая для оценки качества результатов полученных предсказаний.
9. На тренировчных данных обучено несколько моделей. Определена лучшая и  наилучшие гиперпараметры. 
10. Модель проверена на тестовых данных. Показатель sMAPE = 9.88